In [3]:
!pip install openprompt

import pandas as pd
from openprompt.data_utils import InputExample
import torch
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score,f1_score,classification_report
from sklearn.model_selection import train_test_split
from openprompt.config import get_config, save_config_to_yaml
from yacs.config import CfgNode
from tqdm.contrib import tenumerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 3.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99


/opt/conda/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5

In [4]:
path=r"/kaggle/input/rs-2012-03"
df=pd.read_csv(os.path.join(path,"RS_2012-03_violations2.csv"))

In [5]:
classes = [ # There are two classes in Sentiment Analysis, one for negative and one for positive
    "racial_slur",
    "self_harm",
    "Homophobia",
    "Incivility",
    "harrassment",
    "No Violations"
]

encoder = LabelEncoder()
df["label"]=encoder.fit_transform(df["violation"])

le_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

print(le_name_mapping)



{'Homophobia': 0, 'Incivility': 1, 'No Violations': 2, 'harrassment': 3, 'racial_slur': 4, 'self_harm': 5}


In [6]:
from openprompt.plms import load_plm
from openprompt.lm_bff_trainer import LMBFFClassificationRunner,ClassificationRunner
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from openprompt.prompts import ManualTemplate

promptTemplate = ManualTemplate(
    text = '{"placeholder":"text_b"}, Which violation is it? {"mask"} ',
    tokenizer = tokenizer,
)

bad_words = {
    "racial_slur": ["nigga","nigger","uncle tom","negro","niggerhead","house slave","monkeyboy"],
    "self_harm": ["kill yourself", "commit suicide"],
    "Homophobia":["lesbo","faggot","fag",'cocksucker'],
    "Incivility":["dickhead","twat","cunt","whore","retard","bitch","asshole","dimwit","bullshit","fuck u","fuck you","motherfuck"],
    "harrassment":["penis","pussy","dick","tits","moron","suck dick","sexy bitch"]
}

In [8]:
bad_set=set()
for k,v in bad_words.items():
    for i in bad_words[k]: 
        bad_set.add(i)
all_set=set()
for i in df["context"]:
   for k in i.split(' '):
       all_set.add(k.lower())
    
no_vio = all_set.difference(bad_set)  
no_vio=list(no_vio)
no_vio=no_vio[0:len(no_vio):5000]

    
 
from openprompt.prompts import ManualVerbalizer
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        "racial_slur": ["nigga","nigger","uncle tom","negro","niggerhead","house slave","monkeyboy"],
        "self_harm": ["kill yourself", "commit suicide"],
        "Homophobia":["lesbo","faggot","fag",'cocksucker'],
        "Incivility":["dickhead","twat","cunt","whore","retard","bitch","asshole","dimwit","bullshit","fuck u","fuck you","motherfuck"],
        "harrassment":["penis","pussy","dick","tits","moron","suck dick","sexy bitch"],
        "No Violations": no_vio
    },
    tokenizer = tokenizer,
)

In [9]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
    freeze_plm=False,
    plm_eval_mode=False
)


In [11]:
data_train , data_val = train_test_split(df,train_size=0.75,random_state=2018,stratify=df['label'])
#data_train,data_val = train_test_split(data_train_val,train_size=0.25,random_state=2018,stratify=data_train_val['label'])

def InputExampleConverter(df): 
    dataset=[]
    count=0
    violation_sentence = df["sentence"]
    label=df["label"]
    context=df["context"]
    print("Creating Dataset")     
    for i in violation_sentence: 
        dataset.append(InputExample(guid=count,text_a=context[count],text_b=i,label=label[count]))
        count=count+1
    return(dataset)
data_train=InputExampleConverter(data_train.reset_index())
data_val=InputExampleConverter(data_val.reset_index())
#data_test=InputExampleConverter(data_test.reset_index())    

        


from openprompt import PromptDataLoader
data_loader_train = PromptDataLoader(
    dataset = data_train,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    shuffle=True
)

data_loader_val = PromptDataLoader(
    dataset = data_val,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    shuffle=True
)
'''
data_loader_test = PromptDataLoader(
    dataset = data_test,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    shuffle=True
)
'''

Creating Dataset
Creating Dataset


tokenizing: 28it [00:00, 265.86it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1626 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 45762it [02:16, 335.30it/s]
tokenizing: 15254it [00:44, 339.70it/s]


'\ndata_loader_test = PromptDataLoader(\n    dataset = data_test,\n    tokenizer = tokenizer,\n    template = promptTemplate,\n    tokenizer_wrapper_class=WrapperClass,\n    shuffle=True\n)\n'

In [12]:
predictions=[]

use_cuda = True
if use_cuda:
    promptModel=  promptModel.cuda()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
optimizer = AdamW(promptModel.parameters(), lr=1e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:

for epoch in range(5):
    tot_loss = 0
    prev_score=0
    val_score=[]
    test_score=[]
    for step, inputs in tenumerate(data_loader_train):
        if use_cuda:
            inputs = inputs.to(device) 
        #print(step)   
        logits = promptModel(inputs)
        logits=logits.to(device)
        labels = inputs['label']
        labels=labels.to(device)
        loss = loss_func(logits,labels.type(torch.LongTensor).to(device))
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step %5000 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)
            
            try: 
                torch.save({

            'epoch': epoch,

            'model_state_dict': promptModel.state_dict(),

            'optimizer_state_dict': optimizer.state_dict(),

            'loss': loss,

            },os.path.join(path,"violation_type_model_chkpt.pt"))
                
            except:
                continue
    allpreds = []
    alllabels = []
    print("getting val score after epoch "+ str(epoch))
    for step, inputs in tenumerate(data_loader_val):
        if use_cuda:
            inputs = inputs.cuda()
        logits = promptModel(inputs)
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

    print(f1_score(alllabels,allpreds,average="micro" ))
    print(classification_report(alllabels,allpreds))  
    
    

  0%|          | 0/45762 [00:00<?, ?it/s]

Epoch 0, average loss: 3.2978603839874268
Epoch 0, average loss: 0.42030673310039984
Epoch 0, average loss: 0.3243988596719969
Epoch 0, average loss: 0.2903059456407207
Epoch 0, average loss: 0.2650684100013636
Epoch 0, average loss: 0.2492117375109385
Epoch 0, average loss: 0.23413849382015697
Epoch 0, average loss: 0.22437012842782814
Epoch 0, average loss: 0.21666976985066996
Epoch 0, average loss: 0.20970070878347657
getting val score after epoch 0


  0%|          | 0/15254 [00:00<?, ?it/s]

0.9542415104234955
              precision    recall  f1-score   support

           0       0.93      0.74      0.82       280
           1       0.94      0.95      0.94      4311
           2       0.99      0.96      0.97      8460
           3       0.89      0.97      0.93      1954
           4       0.85      0.93      0.89       194
           5       0.88      0.84      0.86        55

    accuracy                           0.95     15254
   macro avg       0.91      0.90      0.90     15254
weighted avg       0.96      0.95      0.95     15254



  0%|          | 0/45762 [00:00<?, ?it/s]

...

0.9566671037105021
              precision    recall  f1-score   support

           0       0.78      0.97      0.87       280
           1       0.94      0.94      0.94      4311
           2       0.99      0.96      0.98      8460
           3       0.89      0.98      0.93      1954
           4       0.87      0.90      0.88       194
           5       0.96      0.80      0.87        55

    accuracy                           0.96     15254
   macro avg       0.90      0.93      0.91     15254
weighted avg       0.96      0.96      0.96     15254



  0%|          | 0/45762 [00:00<?, ?it/s]

Epoch 4, average loss: 1.0430757811263902e-05
Epoch 4, average loss: 0.11976519109586889
Epoch 4, average loss: 0.1214190014570449
Epoch 4, average loss: 0.11832309702974035
Epoch 4, average loss: 0.12262858790390094
Epoch 4, average loss: 0.12397536589811843
Epoch 4, average loss: 0.123409029012607
Epoch 4, average loss: 0.12313569956512063
Epoch 4, average loss: 0.12165101173647781
Epoch 4, average loss: 0.12322537172044574
getting val score after epoch 4


  0%|          | 0/15254 [00:00<?, ?it/s]

0.9520781434377869
              precision    recall  f1-score   support

           0       0.81      0.93      0.86       280
           1       0.89      0.98      0.94      4311
           2       0.99      0.96      0.97      8460
           3       0.95      0.89      0.92      1954
           4       0.92      0.81      0.86       194
           5       0.94      0.84      0.88        55

    accuracy                           0.95     15254
   macro avg       0.92      0.90      0.91     15254
weighted avg       0.95      0.95      0.95     15254



In [ ]:
torch.save(promptModel,os.path.join(path,"violation_detector.pt"))